In [ ]:
import os
from detectron2.data.datasets import register_coco_instances
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.data import MetadataCatalog
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer

import matplotlib.pyplot as plt

import random
import cv2

In [ ]:
ROOT = '/net/birdstore/Active_Atlas_Data/data_root/'
DATA = os.path.join(ROOT, 'brains_info', 'masks', 'structures', 'detectron')
train_data = os.path.join(DATA, 'train')
train_json = os.path.join(DATA, 'structure_training.json')
register_coco_instances(f"structure_train", {}, train_json, train_data)

test_data = os.path.join(ROOT, 'pipeline_data/DK37/preps/C1/thumbnail_aligned')
test_json = os.path.join(DATA, 'DK37_testing.json')
register_coco_instances(f"structure_test", {}, test_json, test_data)

In [ ]:
%%time
cfg = get_cfg()
cfg.MODEL.DEVICE = "cpu"
# Got 0 with mask at 30%
#cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
#cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
# got got several rectangles with this one at 25%
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo
cfg.DATASETS.TRAIN = ("structure_train",)
cfg.DATASETS.TEST = ()
cfg.DATALOADER.NUM_WORKERS = 2
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
#cfg.SOLVER.MAX_ITER = 10
cfg.SOLVER.WARMUP_ITERS = 10
cfg.SOLVER.MAX_ITER = 15 #adjust up if val mAP is still rising, adjust down if overfit
#cfg.SOLVER.STEPS = (10, 15)
cfg.SOLVER.GAMMA = 0.05
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 6
cfg.OUTPUT_DIR = os.path.join(DATA, 'output')
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.18
cfg.DATASETS.TEST = ("structure_test", )
predictor = DefaultPredictor(cfg)
test_metadata = MetadataCatalog.get("structure_test")
test_path = os.path.join(ROOT, 'pipeline_data/DK37/preps/C1/thumbnail_aligned')
test_file = os.path.join(test_path, '270.tif')
im = cv2.imread(test_file)
outputs = predictor(im)
v = Visualizer(im[:, :, ::-1],
               metadata=test_metadata, 
               scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.figure(figsize = (14, 10))
#plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1], cv2.COLOR_BGR2RGB))
plt.imshow(v.get_image())
plt.show()